In [1]:
import numpy as np
import os
import torch
from torch import nn, optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

Using device cuda:0
GeForce GTX 1660 SUPER
P106-100


In [2]:
from torch.utils.data import Dataset, DataLoader
from data.MinicityDataset import TrafficDataset

dbpath = '/home/rong/disk/database/minicity.db'
bs = 12
train_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = True),
                         batch_size = bs, shuffle=True, drop_last = True)
test_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = False),
                          batch_size = bs, shuffle=True, drop_last = True)

# batch = next(iter(train_loader))
# batch['data']

In [3]:
from models.minicity import convVAE

X_dim = 4
traj_size = 25 * 4
z_dim = 50
cnn_out_size = 300
cond_out_size = 300

model = convVAE(sample_size = X_dim, 
                  traj_size = traj_size,
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)

checkpoint = torch.load('checkpoints/minicity_attention_7.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
print(model)

convVAE(
  (condnn): CondNN(
    (cnn): Conv3d(
      (adap_pool): AdaptiveAvgPool3d(output_size=(25, 200, 200))
      (conv_layer1): Sequential(
        (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer2): Sequential(
        (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer5): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
      (adap_pool2): AdaptiveAvg

In [28]:
from utils.Minicity import plotData, plotOrientSpeed, plotAlpha
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
viz_idx = 8753 #弯道
# viz_idx = 143300 #弯道
# viz_idx = 5107 #直行
# viz_idx = 2984 #转盘
# viz_idx = 101538 #转盘
# viz_idx = 153309 #阻塞
# viz_idx = 109779 #变道

print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ[:, 200:600, 200:600]        
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
adap_pool = nn.AdaptiveAvgPool3d((25,200, 200))
occ = adap_pool(occ)
occ = occ.to(device)
traj = torch.from_numpy(batch["traj"]).to(device)
print(traj.shape)
data = torch.from_numpy(batch["data"]).to(device)
egoid = batch["egoid"]
print(traj.shape)
time_stamp = batch["timeprob"]
print("time_step: ", time_stamp)

with torch.no_grad():
    model.eval()
    y_viz = torch.randn(1,4).to(device)
    for i in range(0, 10):
        num_viz = 12
        y_viz_p, alpha = model.inference(startgoal.expand(num_viz, X_dim * 2).to(device), traj.expand(num_viz, 25, 4),
                                occ.expand(num_viz, 1, -1, -1, -1).to(device), num_viz)
        torch.cuda.empty_cache()
        y_viz = torch.cat((y_viz_p, y_viz), dim = 0)

y_viz=y_viz.cpu().detach().numpy()*50
occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy() * 50
print("start", startgoal[0:4])
print("goal", startgoal[4:8])

data=data.cpu().detach().numpy() * 50
alpha=alpha.cpu().detach().numpy()
torch.cuda.empty_cache()
# from utils.NarrowPassage import plotCondition, plotSample, plotSpeed, plotSampleAttention

%matplotlib
# from utils.HighWay import plotData, plotOrientSpeed, plotAlpha
    
y_viz=y_viz[:-1]
plotData(occ, startgoal, y_viz)
plotOrientSpeed(startgoal, y_viz)
# plotAlpha(alpha)

8753
torch.Size([25, 4])
torch.Size([25, 4])
time_step:  1276.0
start [ 0.      0.     -1.4532  9.86  ]
goal [ 16.669998 -20.279999  -0.3976    11.98    ]
Using matplotlib backend: Qt5Agg
[  0.         0.        -1.4532     9.86      16.669998 -20.279999
  -0.3976    11.98    ]
155.56666056315106 167.5999959309896


In [5]:
plotAlpha(alpha)

In [6]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
#  
print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ[:, 200:600, 200:600]        
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
adap_pool = nn.AdaptiveAvgPool3d((25,200, 200))
occ = adap_pool(occ)
# adap_pool = nn.AdaptiveAvgPool3d((25,100, 600))
# occ = adap_pool(occ)
occ = occ.to(device)
data = torch.from_numpy(batch["data"]).to(device)

occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy() * 50
data=data.cpu().detach().numpy() * 50
torch.cuda.empty_cache()

plotData(occ, startgoal, data)

32488
[  0.         0.        -1.5708    10.2        0.       -20.849998
  -1.5708    13.      ]
100.0 169.499994913737


# commonroad collision

In [29]:
import os
import matplotlib.pyplot as plt

from commonroad.common.file_reader import CommonRoadFileReader
from commonroad_cc.visualization.draw_dispatch import draw_object

file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/minicity.cr.xml'
# file_path = "scenarios/commonroad_data/ZAM_Tutorial-1_2_T-1.xml"
# file_path = "/home/rong/commonroad/commonroad-search-master/GSMP/tools/commonroad-road-boundary/scenarios/GER_Ffb_2.xml"
scenario, _ = CommonRoadFileReader(file_path).open()

In [8]:
# plot the scenario
plt.figure(figsize=(25, 10))
draw_object(scenario)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()

In [9]:
from commonroad.scenario.scenario import Scenario
from commonroad.scenario.trajectory import State as StateTupleFactory
from commonroad.scenario.obstacle import StaticObstacle, ObstacleType, Obstacle
import numpy as np
from commonroad.geometry.shape import Polygon, ShapeGroup, Circle
from commonroad_cc.collision_detection.pycrcc_collision_dispatch import create_collision_checker, create_collision_object
import pycrcc

carcc = create_collision_checker(scenario)

from scenarios.commonroad_road_boundary.construction import construct

%matplotlib

build = ['section_triangles']
boundary = construct(scenario, build, ['section_triangles'], ['plot'])
# boundary = construct(scenario, build, ['section_triangles'], [])

road_boundary_shape_list = list()
initial_state = None
for r in boundary['section_triangles'].unpack():
    initial_state = StateTupleFactory(position=np.array([0, 0]), orientation=0.0, time_step=0)
    p = Polygon(np.array(r.vertices()))
    road_boundary_shape_list.append(p)
road_bound = StaticObstacle(obstacle_id=scenario.generate_object_id(), obstacle_type=ObstacleType.ROAD_BOUNDARY,
                            obstacle_shape=ShapeGroup(road_boundary_shape_list), initial_state=initial_state)

roadcc = pycrcc.CollisionChecker()
roadcc.add_collision_object(create_collision_object(road_bound))

Using matplotlib backend: Qt5Agg


In [10]:
import numpy as np
from commonroad.scenario.trajectory import State, Trajectory
from commonroad.prediction.prediction import TrajectoryPrediction
from commonroad.geometry.shape import Rectangle
from commonroad.scenario.obstacle import StaticObstacle
from commonroad_cc.collision_detection.pycrcc_collision_dispatch import create_collision_object

# # create a trajectory for the ego vehicle starting at time step 0
time_start = 0
position = np.array([[153, 147]])
orientation = [0]
state_list = list()
for k in range(0, len(position)):
    state_list.append(State(**{'position': position[k], 'orientation':orientation[k]}))
trajectory = Trajectory(time_start, state_list)
# create the shape of the ego vehicle
shape = Rectangle(length=3.9, width=1.9)
# # create a TrajectoryPrediction object consisting of the trajectory and the shape of the ego vehicle
traj_pred = TrajectoryPrediction(trajectory=trajectory, shape=shape)
# create a collision object using the trajectory prediction of the ego vehicle
co = create_collision_object(traj_pred)

# test the trajectory of the ego vehicle for collisions
print('Collision between the trajectory of the ego vehicle and objects in the environment: ', carcc.collide(co))
print('Collision between the trajectory of the ego vehicle and road: ', roadcc.collide(co))
print('Vallid', roadcc.collide(co) and not carcc.collide(co))

Collision between the trajectory of the ego vehicle and objects in the environment:  False
Collision between the trajectory of the ego vehicle and road:  True
Vallid True


# ompl

In [11]:
from ompl import base as ob
from ompl import control as oc
from ompl import app as oa

class ValidityChecker(ob.StateValidityChecker):
    def __init__(self,  si, ccc, ccr, time_stamp):
        super(ValidityChecker, self).__init__(si)
        self.ccc = ccc
        self.ccr = ccr
        self.time_stamp = int(time_stamp)
        self.count = 0
        self.collision_count = 0
        self.states_ok = []
        self.states_bad = []
        print("")
        # Returns whether the given state's position overlaps the
        # circular obstacle
    def isValid(self, state):
        time_start = self.time_stamp
        position = np.array([[state[0].getX(), state[0].getY()]])
        orientation = [state[0].getYaw()]
        state_list = list()
        for k in range(0, len(position)):
            state_list.append(State(**{'position': position[k], 'orientation':orientation[k]}))
        trajectory = Trajectory(time_start, state_list)
        shape = Rectangle(length=3.9, width=1.9)
        traj_pred = TrajectoryPrediction(trajectory=trajectory, shape=shape)
        co = create_collision_object(traj_pred)

        result = self.ccr.collide(co) and not self.ccc.collide(co)
        self.count += 1
#         print("count", self.count)
        if not result:
            self.collision_count += 1
#             print("collision", self.collision_count)
#             self.states_bad.append(sample)
#         else:
#             self.states_ok.append(sample)
        return result

car = oa.DynamicCarPlanning()
mychecker = ValidityChecker(car.getSpaceInformation(), carcc, roadcc, 0)
print(mychecker)

In [12]:
import sys
from os.path import abspath, dirname, join

from ompl import base as ob
from ompl import control as oc
from ompl import app as oa


import matplotlib.pyplot as plt

def plt_ompl_result(path, time_step):
    if path:
        fig1 = plt.figure(figsize=(10, 6), dpi=80)
        ax1 = fig1.add_subplot(111, aspect='equal')   
        draw_params = {}
        draw_params['time_begin'] = time_step
        draw_object(scenario, draw_params=draw_params)

        path.interpolate(); # uncomment if you want to plot the path

        states = path.getStates()
        for state in states:
            state = state[0]
            plt.scatter(state.getX(), state.getY(), color="green", s=250, edgecolors='black')  # path
        #     for state in collisionchecker.states_ok:
        #         plt.scatter(state[0], state[1], color="green", s=100, edgecolors='green')  # free sample
        #     for state in collisionchecker.states_bad:
        #         plt.scatter(state[0], state[1], color="red", s=100, edgecolors='red')  # collision sample
        # plt.scatter(start().getX(), start().getY(), color="blue", s=250, edgecolors='black')  # init
        # plt.scatter(goal().getX(), goal().getY(), color="red", s=250, edgecolors='black')  # goal

        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()
    else:
        print("no path to plot")
    
def dynamicCarDemo(setup):
    print("\n\n***** Planning for a %s *****\n" % setup.getName())
    # plan for dynamic car in SE(2)
    stateSpace = setup.getStateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(100)
    bounds.setHigh(200)
    stateSpace.getSubspace(0).setBounds(bounds)
    bounds.setLow(0, -3)
    bounds.setHigh(0, 15)
    bounds.setLow(1, -0.52)
    bounds.setHigh(1, 0.52)
    stateSpace.getSubspace(1).setBounds(bounds)

    # define start state
    start = ob.State(stateSpace)
    start[0] = 153
    start[1] = 153
    start[2] = -1.45
    start[3] = 9.86
    start[4] = 0

    # define goal state
    goal = ob.State(stateSpace)
    goal[0] = 171.6
    goal[1] = 123.3
    goal[2] = -0.39
    goal[3] = 10
    goal[4] = 0
    print(start, goal)
    # set the start & goal states
    setup.setStartAndGoalStates(start, goal, .5)
    setup.getSpaceInformation().setStateValidityChecker(mychecker)

#     print(setup.getSpaceInformation().getStateValidityChecker())
    planner = oc.RRT(setup.getSpaceInformation())

    
    setup.setPlanner(planner)
#     print(planner.getSpaceInformation().getStateValidityChecker())
    planner.setProblemDefinition(setup.getProblemDefinition())
#     setup.getPlanner().setup()
    # try to solve the problem
    plan_res =  planner.solve(40)
    path = planner.getProblemDefinition().getSolutionPath()
    if plan_res:
#         path.interpolate(); # uncomment if you want to plot the path
        print(path.printAsMatrix())
        if plan_res.asString() != 'Exact solution':
            print("Solution is approximate. Distance to actual goal is %g" %
                  planner.getProblemDefinition().getSolutionDifference())
    else:
        print("plan fail")
    return path


car = oa.DynamicCarPlanning()
car.getSpaceInformation().setStateValidityChecker(mychecker)
car.getSpaceInformation().setPropagationStepSize(0.3)
car.getSpaceInformation().setStateValidityCheckingResolution(0.0001)

# path = dynamicCarDemo(car)
# plt_ompl_result(path)

In [13]:
def checkpath(path, checker):
    states = path.getStates()
    for state in states:
        print(state[0].getX(), state[0].getY())
        if not checker.isValid(state):
            print("valid False")

In [179]:
path = dynamicCarDemo(car)
plt_ompl_result(path, 0)



***** Planning for a Dynamic car *****

Compound state [
Compound state [
RealVectorState [153 153]
SO2State [-1.45]
]
RealVectorState [9.86 0]
]
 Compound state [
Compound state [
RealVectorState [171.6 123.3]
SO2State [-0.39]
]
RealVectorState [10 0]
]

153 153 -1.45 9.86 0 0 0 0
153.374 150.043 -1.43465 10.0085 0.0102745 0.494875 0.0342483 0.3
153.843 147.077 -1.38888 10.0165 0.0201997 0.0267254 0.0330841 0.3

Solution is approximate. Distance to actual goal is 30.1831


In [180]:
checkpath(path, mychecker)

153.0 153.0
153.3742774840054 150.0433573248753
153.84254515285346 147.07660800549021


In [56]:
mychecker.count

442994

In [57]:
mychecker.collision_count

133693

In [35]:
def dynamicCarMinicity(setup, basepose, startgoal):
    print("\n\n***** Planning for a %s *****\n" % setup.getName())
    # plan for dynamic car in SE(2)
    stateSpace = setup.getStateSpace()
    sizebound = 50
    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(0, basepose[0] - sizebound)
    bounds.setHigh(0, basepose[0] + sizebound)
    bounds.setLow(1, basepose[1]- sizebound)
    bounds.setHigh(1, basepose[1] + sizebound)
    stateSpace.getSubspace(0).setBounds(bounds)
    bounds.setLow(0, -7)
    bounds.setHigh(0, 15)
    bounds.setLow(1, -0.52)
    bounds.setHigh(1, 0.52)
    stateSpace.getSubspace(1).setBounds(bounds)
    print(setup.getStateSpace().getSubspace(0).getBounds().high[0])

    # define start state
    start = ob.State(stateSpace)
    start[0] = float(startgoal[0])
    start[1] = float(startgoal[1])
    start[2] = float(startgoal[2])
    start[3] = float(startgoal[3])
    start[4] = 0

    print(start)
    # define goal state
    goal = ob.State(stateSpace)
    goal[0] = float(startgoal[4])
    goal[1] = float(startgoal[5])
    goal[2] = float(startgoal[6])
    goal[3] = float(startgoal[7])
    goal[4] = 0
    print(goal)

    # set the start & goal states
    setup.setStartAndGoalStates(start, goal, .5)
    setup.getSpaceInformation().setStateValidityChecker(mychecker)

#     print(setup.getSpaceInformation().getStateValidityChecker())
    planner = oc.RRT(setup.getSpaceInformation())

    
    setup.setPlanner(planner)
#     print(planner.getSpaceInformation().getStateValidityChecker())
    planner.setProblemDefinition(setup.getProblemDefinition())
#     setup.getPlanner().setup()
    # try to solve the problem
#     planner.setNumSamples(30)
    plan_res =  planner.solve(1)
    path = planner.getProblemDefinition().getSolutionPath()
    if plan_res:
#         path.interpolate(); # uncomment if you want to plot the path
        print(path.printAsMatrix())
        if plan_res.asString() != 'Exact solution':
            print("Solution is approximate. Distance to actual goal is %g" %
                  planner.getProblemDefinition().getSolutionDifference())
    else:
        print("plan fail")
    return path, planner

In [30]:
import math
def warp2pi(ang):
    while ang > math.pi:
        ang -= math.pi
    while ang < -math.pi:
        ang += math.pi
    return ang

def getproblem(egoid, time_stamp, startgoal):
    ego = scenario.obstacle_by_id(egoid)
    basepose = ego.occupancy_at_time(int(time_stamp)).shape.center

    startgoal[0] += basepose[0]
    startgoal[1] += basepose[1]
    startgoal[2] = warp2pi(startgoal[2])
    startgoal[4] += basepose[0]
    startgoal[5] += basepose[1]
    startgoal[6] = warp2pi(startgoal[6])
    
    return basepose, startgoal

ego = scenario.obstacle_by_id(egoid)
basepose, startgoal = getproblem(egoid, time_stamp, startgoal)

In [36]:
car = oa.DynamicCarPlanning()
car.getSpaceInformation().setPropagationStepSize(0.3)
car.getSpaceInformation().setStateValidityCheckingResolution(0.00001)
mychecker = ValidityChecker(car.getSpaceInformation(), carcc, roadcc, 0)
car.getSpaceInformation().setStateValidityChecker(mychecker)
# scenario.remove_obstacle(ego)
path, planner = dynamicCarMinicity(car, basepose, startgoal)
# scenario.add_objects(ego)
print("Count", mychecker.count)
print("Collision", mychecker.collision_count)




***** Planning for a Dynamic car *****

204.89
Compound state [
Compound state [
RealVectorState [154.89 153.59]
SO2State [-1.4532]
]
RealVectorState [9.86 0]
]

Compound state [
Compound state [
RealVectorState [171.56 133.31]
SO2State [-0.3976]
]
RealVectorState [11.98 0]
]

154.89 153.59 -1.4532 9.86 0 0 0 0
157.14 142.236 -1.21814 9.47862 0.0407737 -0.31782 0.0339781 1.2
159.673 137.068 -1.02638 9.73584 0.0257993 0.428707 -0.0249573 0.6

Solution is approximate. Distance to actual goal is 13.4543
Count 3873
Collision 1189


In [37]:
plt_ompl_result(path, int(time_stamp))

In [46]:
def plttree(planner, startgoal):
    fig1 = plt.figure(figsize=(10, 6), dpi=80)
    ax1 = fig1.add_subplot(111, aspect='equal')
    
    plannerdata = ob.PlannerData(planner.getSpaceInformation())
    planner.getPlannerData(plannerdata)

    print("num edges: ", plannerdata.numEdges())
    print("num vertices: ", plannerdata.numVertices())

    plt.scatter(startgoal[0], startgoal[1], color="blue", s=250, edgecolors='black')  # init
    plt.scatter(startgoal[4], startgoal[5], color="red", s=250, edgecolors='black')  # goal

    num_ver = plannerdata.numVertices()
    for i in range(0, num_ver):
        plt.scatter( plannerdata.getVertex(i).getState()[0].getX(), plannerdata.getVertex(i).getState()[0].getY(), color="green", s=50, edgecolors='black')  # vertice
#         for j in range(i, num_ver):
#             if plannerdata.edgeExists(i, j):
#                 plt.plot([plannerdata.getVertex(i).getState()[0].getX(), plannerdata.getVertex(j).getState()[0].getX()], [plannerdata.getVertex(i).getState()[0].getY(), plannerdata.getVertex(j).getState()[0].getY()], color='gray')

    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

In [47]:
plttree(planner, startgoal)

num edges:  781
num vertices:  782


In [73]:
basepose

array([154.89, 153.59])

In [138]:
startgoal

array([154.89  , 153.59  ,  -1.4532,   9.86  , 171.56  , 133.31  ,
        -0.3976,  11.98  ], dtype=float32)

In [74]:
time_stamp

1276.0